# 3 Pointers Made against advanced_totals.csv

### Import packages

In [16]:
import os
import numpy as np
import pandas as pd
%matplotlib inline
from scipy.stats import pearsonr, zscore
import itertools

pd.set_option("display.max_columns", None)

### Set working directory

In [17]:
# Print working directory
cwd = os.getcwd()
print(f'Directory: {cwd}')

# Change working directory
os.chdir('/Users/tyler/OneDrive/Documents/Python/NBA')

# Print working directory
cwd = os.getcwd()
print(f'Directory: {cwd}')

Directory: C:\Users\tyler\OneDrive\Documents\Python\NBA
Directory: C:\Users\tyler\OneDrive\Documents\Python\NBA


## Exploratory Data Analysis

### Import data

In [18]:
advanced_df = pd.read_csv('backend/data/totals/advanced_totals.csv').drop(['Unnamed: 0'], axis=1)
shooting_df = pd.read_csv('backend/data/totals/game_totals.csv').drop(['Unnamed: 0'], axis=1)
shooting_df = shooting_df[['date', 'visitor', 'home', 'team', '3p']]

In [19]:
# Merge dataframes to have target variable
df = pd.merge(shooting_df, advanced_df, 
              left_on=['date', 'visitor', 'home', 'team'], right_on=['date', 'visitor', 'home', 'team'],
              how='left')

### Basic exploration

In [20]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 38288 entries, 0 to 38287
Data columns (total 19 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   date      38288 non-null  object 
 1   visitor   38288 non-null  object 
 2   home      38288 non-null  object 
 3   team      38288 non-null  int64  
 4   3p        38288 non-null  int64  
 5   ts_perc   38288 non-null  float64
 6   efg_perc  38288 non-null  float64
 7   3par      38288 non-null  float64
 8   ftr       38288 non-null  float64
 9   orb_perc  38288 non-null  float64
 10  drb_perc  38288 non-null  float64
 11  trb_perc  38288 non-null  float64
 12  ast_perc  38288 non-null  float64
 13  stl_perc  38288 non-null  float64
 14  blk_perc  38288 non-null  float64
 15  tov_perc  38288 non-null  float64
 16  usg_perc  38288 non-null  float64
 17  ortg      38288 non-null  float64
 18  drtg      38288 non-null  float64
dtypes: float64(14), int64(2), object(3)
memory usage: 5.8+ MB


In [21]:
# Convert 'date' column to Date object
df['date'] = pd.to_datetime(df['date'])

# Convert 'team' column to Team Name
df['team'] = np.where(df['team'], df['home'], df['visitor'])

# Rename target variable
df = df.rename({'3p': 'target'}, axis=1)

# Define statistics
stats = ['ts_perc', 'efg_perc', '3par', 'ftr', 'orb_perc', 'drb_perc', 'trb_perc', 
         'ast_perc', 'stl_perc', 'blk_perc', 'tov_perc', 'usg_perc', 'ortg', 'drtg']

df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 38288 entries, 0 to 38287
Data columns (total 19 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   date      38288 non-null  datetime64[ns]
 1   visitor   38288 non-null  object        
 2   home      38288 non-null  object        
 3   team      38288 non-null  object        
 4   target    38288 non-null  int64         
 5   ts_perc   38288 non-null  float64       
 6   efg_perc  38288 non-null  float64       
 7   3par      38288 non-null  float64       
 8   ftr       38288 non-null  float64       
 9   orb_perc  38288 non-null  float64       
 10  drb_perc  38288 non-null  float64       
 11  trb_perc  38288 non-null  float64       
 12  ast_perc  38288 non-null  float64       
 13  stl_perc  38288 non-null  float64       
 14  blk_perc  38288 non-null  float64       
 15  tov_perc  38288 non-null  float64       
 16  usg_perc  38288 non-null  float64       
 17  ortg      38

# Dataframe of team's last 15 performances

In [22]:
# Return ten lastest dates team played
def last_15_date(team, date):
    schedule = df[df['team'] == team].sort_values(by='date').reset_index()
    date_index = schedule[schedule['date'] == date].index[0]
    if date_index - 15 < 0:
        return None, None, None, None, None, None, None, None, None, None, None, None, None, None, None
    else:
        date_1, date_2 = schedule.iloc[date_index - 1]['date'], schedule.iloc[date_index - 2]['date']
        date_3, date_4 = schedule.iloc[date_index - 3]['date'], schedule.iloc[date_index - 4]['date']
        date_5, date_6 = schedule.iloc[date_index - 5]['date'], schedule.iloc[date_index - 6]['date']
        date_7, date_8 = schedule.iloc[date_index - 7]['date'], schedule.iloc[date_index - 8]['date']
        date_9, date_10 = schedule.iloc[date_index - 9]['date'], schedule.iloc[date_index - 10]['date']
        date_11, date_12 = schedule.iloc[date_index - 11]['date'], schedule.iloc[date_index - 12]['date']
        date_13, date_14 = schedule.iloc[date_index - 13]['date'], schedule.iloc[date_index - 14]['date']
        date_15 = schedule.iloc[date_index - 15]['date']
        return date_1, date_2, date_3, date_4, date_5, date_6, date_7, date_8, date_9, date_10, date_11, date_12, date_13, date_14, date_15

df['dates'] = df.apply(lambda x: last_15_date(x.team, x.date), axis=1)
df['date_1'], df['date_2'] = df['dates'].apply(lambda x: x[0]), df['dates'].apply(lambda x: x[1])
df['date_3'], df['date_4'] = df['dates'].apply(lambda x: x[2]), df['dates'].apply(lambda x: x[3])
df['date_5'], df['date_6'] = df['dates'].apply(lambda x: x[4]), df['dates'].apply(lambda x: x[5])
df['date_7'], df['date_8'] = df['dates'].apply(lambda x: x[6]), df['dates'].apply(lambda x: x[7])
df['date_9'], df['date_10'] = df['dates'].apply(lambda x: x[8]), df['dates'].apply(lambda x: x[9])
df['date_11'], df['date_12'] = df['dates'].apply(lambda x: x[10]), df['dates'].apply(lambda x: x[11])
df['date_13'], df['date_14'] = df['dates'].apply(lambda x: x[12]), df['dates'].apply(lambda x: x[13])
df['date_15'] = df['dates'].apply(lambda x: x[14])

In [23]:
# X and y column names to merge on
y_cols = df.columns
x_cols = ['date', 'team'] + stats

last_15_games = df[y_cols]
X = df[x_cols]

# Dataframe of target (3pt made by each team) and of variables (last 15 games stats for each team)
dates = ['_1', '_2', '_3', '_4', '_5', '_6', '_7', '_8', '_9', '_10', '_11', '_12', '_13', '_14', '_15']
for date in dates:
    last_15_games = pd.merge(last_15_games, X, left_on=['date' + date, 'team'], right_on=['date', 'team'], how='left', suffixes=('', date))

In [24]:
# Calculate z-score
def z_score(value, mean, std):
    return (value - mean) / std

In [25]:
# Calculate perc difference
def perc_diff(value, mean):
    return (value - mean) / mean

### Last 15 Performances (Unweighted)

In [26]:
dates = ['_1', '_2', '_3', '_4', '_5', '_6', '_7', '_8', '_9', '_10', '_11', '_12', '_13', '_14', '_15']
cols = ['date', 'visitor', 'home', 'team', 'target'] + \
    [tup[0] + tup[1] for tup in list(itertools.product(stats, dates))]

last_15_games_unweighted = last_15_games[cols].copy()

# Calculate mean for each stat over a team's last performance
for stat in stats:
    last_15_games_unweighted[stat] = 0
    for date in dates:
        last_15_games_unweighted[stat] = last_15_games_unweighted[stat] + last_15_games_unweighted[stat + date]
    
    last_15_games_unweighted[stat] = last_15_games_unweighted[stat] / len(dates)
    
# Calculate standard deviation for each stat over a team's performance
for stat in stats:
    last_15_games_unweighted[stat + '_std'] = 0
    for date in dates:
        last_15_games_unweighted[stat + '_std'] = last_15_games_unweighted[stat + '_std'] + \
                                                    ((last_15_games_unweighted[stat + date] - last_15_games_unweighted[stat]) ** 2)
    
    last_15_games_unweighted[stat + '_std'] = last_15_games_unweighted[stat + '_std'] / len(dates)
    last_15_games_unweighted[stat + '_std'] = last_15_games_unweighted[stat + '_std'] ** .5

# Feature engineer trends
for stat in stats:
    last_15_games_unweighted[stat + '_trend'] = 0
    for date in dates[:10]:
        last_15_games_unweighted[stat + '_trend'] = last_15_games_unweighted[stat + '_trend'] + \
                                        z_score(last_15_games_unweighted[stat + date], last_15_games_unweighted[stat], last_15_games_unweighted[stat + '_std']).fillna(0)
    
    last_15_games_unweighted[stat + '_trend'] = last_15_games_unweighted[stat + '_trend'] / len(dates[:10])

last_15_games_unweighted = last_15_games_unweighted.groupby(['date', 'visitor', 'home']).aggregate(['mean', 'sum'])

last_15_game_cols = [col 
                    for col in last_15_games_unweighted.columns
                    if (col[0] == 'target' and col[1] == 'sum') or \
                       (col[0] in stats and col[1] == 'sum' and '_perc' not in col[0]) or \
                       (col[0] in stats and col[1] == 'mean' and '_perc' in col[0]) or \
                       ('_trend' in col[0] and col[1] == 'sum')]

last_15_games_unweighted = last_15_games_unweighted[last_15_game_cols].dropna(axis=0)
last_15_games_unweighted.columns = [col[0] for col in last_15_games_unweighted.columns]
last_15_games_unweighted.head()

target   ts_perc  efg_perc  \
date       visitor         home                                                 
2006-11-28 Indiana Pacers  Portland Trail Blazers      13  0.546800  0.492600   
           New York Knicks Chicago Bulls                9  0.536333  0.482333   
2006-11-29 Indiana Pacers  Golden State Warriors       18  0.541533  0.500067   
           New York Knicks Cleveland Cavaliers         13  0.535333  0.483600   
           Orlando Magic   Seattle SuperSonics          8  0.549067  0.504367   

                                                       3par       ftr  \
date       visitor         home                                         
2006-11-28 Indiana Pacers  Portland Trail Blazers  0.201067  0.369000   
           New York Knicks Chicago Bulls           0.192533  0.389600   
2006-11-29 Indiana Pacers  Golden State Warriors   0.502667  0.658400   
           New York Knicks Cleveland Cavaliers     0.185733  0.393333   
           Orlando Magic   Seattle SuperSonics     0.391067  0.723800   

                                                    orb_perc   drb_perc  \
date       visitor         home                                           
2006-11-28 Indiana Pacers  Portland Trail Blazers  25.980000  71.826667   
           New York Knicks Chicago Bulls           29.100000  74.733333   
2006-11-29 Indiana Pacers  Golden State Warriors   27.246667  68.430000   
           New York Knicks Cleveland Cavaliers     28.060000  74.780000   
           Orlando Magic   Seattle SuperSonics     29.463333  72.186667   

                                                    trb_perc   ast_perc  \
date       visitor         home                                           
2006-11-28 Indiana Pacers  Portland Trail Blazers  48.606667  51.186667   
           New York Knicks Chicago Bulls           51.486667  49.206667   
2006-11-29 Indiana Pacers  Golden State Warriors   47.986667  63.093333   
           New York Knicks Cleveland Cavaliers     51.073333  49.380000   
           Orlando Magic   Seattle SuperSonics     50.903333  51.423333   

                                                   stl_perc   blk_perc  \
date       visitor         home                                          
2006-11-28 Indiana Pacers  Portland Trail Blazers  7.366667   6.780000   
           New York Knicks Chicago Bulls           6.966667   5.466667   
2006-11-29 Indiana Pacers  Golden State Warriors   8.460000  10.166667   
           New York Knicks Cleveland Cavaliers     7.373333   5.493333   
           Orlando Magic   Seattle SuperSonics     8.660000   7.746667   

                                                    tov_perc  usg_perc  \
date       visitor         home                                          
2006-11-28 Indiana Pacers  Portland Trail Blazers  14.386667     100.0   
           New York Knicks Chicago Bulls           14.866667     100.0   
2006-11-29 Indiana Pacers  Golden State Warriors   14.580000     100.0   
           New York Knicks Cleveland Cavaliers     15.126667     100.0   
           Orlando Magic   Seattle SuperSonics     15.363333     100.0   

                                                         ortg        drtg  \
date       visitor         home                                             
2006-11-28 Indiana Pacers  Portland Trail Blazers  106.546667  113.226667   
           New York Knicks Chicago Bulls           105.100000  107.000000   
2006-11-29 Indiana Pacers  Golden State Warriors   212.833333  210.560000   
           New York Knicks Cleveland Cavaliers     103.966667  107.106667   
           Orlando Magic   Seattle SuperSonics     215.006667  211.246667   

                                                   ts_perc_trend  \
date       visitor         home                                    
2006-11-28 Indiana Pacers  Portland Trail Blazers      -0.097118   
           New York Knicks Chicago Bulls               -0.033644   
2006-11-29 Indiana Pacers  Golden State Warriors        0.224904   
 

## Correlations

In [27]:
corr_df = pd.DataFrame()

# Correlations for last 15 game stats vs 3pt made (unweighted)
for col in last_15_games_unweighted:
    corr_p = pearsonr(last_15_games_unweighted['target'], last_15_games_unweighted[col])
    row = {'stat': col, 'corr': round(corr_p[0], 2), 'p-value': round(corr_p[1], 2)}
    corr_df = corr_df.append(row, ignore_index=True)
    
# Print correlations
corr_df = corr_df[corr_df['p-value'] < .05].drop(['p-value'], axis=1).sort_values(['corr'], axis=0, ascending=False)
corr_df

c:\python38\lib\site-packages\scipy\stats\stats.py:4023: PearsonRConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(PearsonRConstantInputWarning())


,corr,stat
0,1.00,target
3,0.72,3par
2,0.51,efg_perc
6,0.45,drb_perc
1,0.45,ts_perc
13,0.29,ortg
14,0.25,drtg
10,0.22,blk_perc
8,0.18,ast_perc
17,0.04,3par_trend
